In [1]:
!unzip -j /content/utils.zip -d.

Archive:  /content/utils.zip
  inflating: ./weather_classification_data.csv  
  inflating: ./data_scaled_labeled_check.csv  


In [2]:
import pandas as pd
data = pd.read_csv("/content/weather_classification_data.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


In [4]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
12422,11.0,93,8.0,61.0,partly cloudy,1010.05,2,Spring,2.0,coastal,Rainy
8898,23.0,60,12.0,44.0,overcast,1007.16,4,Spring,7.5,inland,Cloudy
6611,-9.0,71,10.0,84.0,overcast,989.79,0,Winter,3.0,inland,Snowy
242,24.0,27,2.0,12.0,partly cloudy,1015.66,7,Spring,6.0,inland,Sunny
12560,15.0,74,18.0,90.0,partly cloudy,1006.95,2,Winter,5.0,mountain,Rainy


In [5]:
# START CODE HERE
object_columns = data.select_dtypes(include='object').columns.tolist()
non_object_columns = data.select_dtypes(exclude='object').columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


In [6]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


In [7]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE

column_transformer = ColumnTransformer(
    transformers=[
        ('ordinal_encoder', OrdinalEncoder(), object_columns),
        ('standard_scaler', StandardScaler(), non_object_columns)
    ],
    remainder='passthrough'
)

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)

# END CODE HERE

all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

In [8]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


In [9]:
data_scaled_labeled = data_scaled_labeled.rename(columns={'Weather Type': 'label'})

X = data_scaled_labeled.drop('label', axis=1)
y = data_scaled_labeled['label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Failed
Test Failed


In [11]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
3958,3.0,1.0,1.0,0.452811,0.410487,0.241415,-0.270600,-0.131939,-0.520104,0.752538
2239,2.0,1.0,0.0,0.510330,0.063841,0.169040,1.012843,0.235014,-1.038715,-0.730539
3608,2.0,3.0,2.0,0.395293,-0.778015,-1.350840,-0.270600,-0.074678,-0.520104,0.455922
6848,3.0,2.0,2.0,-0.007338,0.212404,-0.988963,-0.145386,-0.139466,-0.520104,0.010999
6119,2.0,3.0,2.0,-1.617860,1.351385,-1.133714,0.073739,-0.185168,-0.779410,-0.878846


In [12]:
# START CODE HERE
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
# END CODE HERE

print("Accuracy:", score * 100)


Accuracy: 91.51515151515152


In [13]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


In [14]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
4111,2.0,3.0,1.0,-1.100192,0.014320,-0.554712,1.388485,-0.216352,-1.038715,-0.285616
10607,0.0,0.0,1.0,0.165218,-1.817954,-0.120461,-1.366221,0.129902,0.257813,0.307615
7372,3.0,0.0,2.0,-0.007338,-0.579931,0.458541,-1.147097,0.223992,-0.260799,0.752538
11786,3.0,0.0,2.0,0.222737,1.202822,-0.265211,1.388485,0.313244,-0.779410,-0.582231
12227,2.0,2.0,1.0,0.107699,-0.134243,-1.350840,0.637201,-2.110262,2.332257,4.015305


In [15]:
y = model.predict(input)
y

array([2., 3., 0., 1., 3.])